# Essential Model Documentation - Post-processing

Generate RDF graphs and D3/Graphology JSON visualizations.

In [1]:
import json, os, glob, re
from collections import Counter, defaultdict

try:
    import cmipld
    prefix = cmipld.prefix()
    cmipld.map_current(prefix)
    from rdflib import Graph as RGraph, Namespace, URIRef
    from rdflib.namespace import NamespaceManager
    HAS_RDF = True
except ImportError:
    HAS_RDF = False
    prefix = 'emd'

# Extract colors from CSS
def get_colors():
    colors = {'primary': '#2196f3', 'primary_light': '#bbdefb', 'primary_dark': '#1976d2'}
    try:
        css = open('docs/stylesheets/custom.css').read()
        for key in ['primary', 'primary-light', 'primary-dark']:
            if m := re.search(f'--emd-{key}:\\s*([^;]+);', css):
                colors[key.replace('-', '_')] = m.group(1).strip()
    except: pass
    return colors

COLORS = get_colors()
print(f'Colors: {COLORS}')
print(f'Prefix: {prefix}')

Initializing LDR client...
LDR client initialized.
Initializing LDR client...
LDR client initialized.
Added mapping: https://emd.mipcvs.dev/ -> /Users/daniel.ellis/WIPwork/Essential-Model-Documentation/
Colors: {'primary': '#2196f3', 'primary_light': '#bbdefb', 'primary_dark': '#1976d2'}
Prefix: emd


## Process Folders → JSON-LD & RDF

In [2]:
folders = [os.path.dirname(p) for p in glob.glob('*/_context.json')]

# Generate JSON-LD
for folder in folders:
    ctx = json.load(open(f'{folder}/_context.json'))
    ids = [json.load(open(f))['@id'] for f in glob.glob(f'{folder}/*.json') 
           if not os.path.basename(f).startswith('_') and '@id' in json.load(open(f))]
    graph = {'@context': ctx['@context'], '@type': ['Collection'], 'contents': [{'@id': i} for i in ids]}
    with open(f'{folder}/_graph.jsonld', 'w') as f:
        f.write(re.sub(r'\\{\\s+"@id":\\s+"([^"]+)"\\s+\\}', r'{"@id": "\\1"}', json.dumps(graph, indent=2)))
    print(f'✓ {folder}/_graph.jsonld')

# Process RDF
rdf_graphs = {}
if HAS_RDF:
    for v in ['Model', 'ModelFamily', 'ModelComponent', 'ComponentConfig',
              'HorizontalComputationalGrid', 'HorizontalGridCells', 
              'HorizontalSubgrid', 'VerticalComputationalGrid']:
        cmipld.mapping[f'vocab_{v}'] = f'https://emd.mipcvs.dev/docs/vocabularies/{v}/'
    
    for folder in folders:
        try:
            ctx = json.load(open(f'{folder}/_context'))
            data = cmipld.expand(f'{prefix}:{folder}/_graph.jsonld', depth=3)
            g = RGraph()
            g.namespace_manager = NamespaceManager(g)
            for p, u in cmipld.mapping.items(): g.bind(p+':', Namespace(u), replace=True)
            g.parse(data=json.dumps(data), format='json-ld')
            g.serialize(f'{folder}/_graph.ttl', format='turtle')
            rdf_graphs[folder] = g
            print(f'✓ {folder}/_graph.ttl')
        except: pass

print(f'\nProcessed {len(rdf_graphs)} RDF graphs')

✓ model_family/_graph.jsonld
✓ component_config/_graph.jsonld
✓ horizontal_grid_cells/_graph.jsonld
✓ model_component/_graph.jsonld
✓ horizontal_subgrid/_graph.jsonld
✓ model/_graph.jsonld
✓ horizontal_computational_grid/_graph.jsonld
✓ vertical_computational_grid/_graph.jsonld
[Cache HIT] emd:model_family/_graph.jsonld (depth=3)
✓ model_family/_graph.ttl
[Cache HIT] emd:component_config/_graph.jsonld (depth=3)
✓ component_config/_graph.ttl
[Cache HIT] emd:horizontal_grid_cells/_graph.jsonld (depth=3)
✓ horizontal_grid_cells/_graph.ttl
[Cache HIT] emd:model_component/_graph.jsonld (depth=3)
✓ model_component/_graph.ttl
[Cache HIT] emd:horizontal_subgrid/_graph.jsonld (depth=3)
✓ horizontal_subgrid/_graph.ttl
[Cache HIT] emd:model/_graph.jsonld (depth=3)
✓ model/_graph.ttl
[Cache HIT] emd:horizontal_computational_grid/_graph.jsonld (depth=3)
✓ horizontal_computational_grid/_graph.ttl
[Cache HIT] emd:vertical_computational_grid/_graph.jsonld (depth=3)
✓ vertical_computational_grid/_graph

## Generate D3/Graphology JSON

In [7]:
# Extract relationships
def shorten(uri):
    if not isinstance(uri, URIRef): return str(uri)
    for p, ns in cmipld.mapping.items():
        if str(uri).startswith(ns):
            local = str(uri)[len(ns):]
            if 'docs/vocabularies/' in local: local = local.split('/')[-1]
            return f'{p}:{local}'
    return str(uri)

all_rels = []
if HAS_RDF and rdf_graphs:
    for g in rdf_graphs.values():
        for s, p, o in g:
            if 'www.w3.org' in str(p): continue
            if 'mipcvs.dev' in str(s) and 'mipcvs.dev' in str(o):
                all_rels.append((shorten(s), shorten(p), shorten(o)))
    print(f'Extracted {len(all_rels)} relationships\n')

# Generate JSON files
if all_rels:
    # Relationships JSON (entity-level)
    nodes = []
    edges = []
    seen_nodes = set()
    
    for s, p, t in all_rels:
        if s not in seen_nodes:
            nodes.append({'id': s, 'label': s.split('/')[-1], 
                         'color': COLORS['primary'] if s.startswith(f'{prefix}:') else '#808080'})
            seen_nodes.add(s)
        if t not in seen_nodes:
            nodes.append({'id': t, 'label': t.split('/')[-1], 
                         'color': COLORS['primary'] if t.startswith(f'{prefix}:') else '#808080'})
            seen_nodes.add(t)
        edges.append({'source': s, 'target': t, 'label': p, 'color': COLORS['primary_light']})
    
    with open('_d3graph.json', 'w') as f:
        json.dump({'nodes': nodes, 'links': edges}, f, indent=2)
    print(f'✓ _d3graph.json: {len(nodes)} nodes, {len(edges)} edges')
    
    # Structure JSON (folder-level)
    folder = lambda x: x.rsplit('/', 1)[0] if '/' in x else x
    struct_nodes = [{'id': 'root', 'label': 'EMD', 'color': COLORS['primary_dark'], 'size': 30}]
    struct_edges = []
    folder_set = set()
    folder_links = defaultdict(int)
    
    for s, p, t in all_rels:
        sf, tf = folder(s), folder(t)
        if sf and tf:
            for f in [sf, tf]:
                if f not in folder_set and f:
                    folder_set.add(f)
                    struct_nodes.append({
                        'id': f, 'label': f.split(':')[-1],
                        'color': COLORS['primary_light'] if f.startswith(f'{prefix}:') else '#808080',
                        'size': 15
                    })
                    
                    if f.startswith(f'{prefix}:'):
                        struct_edges.append({'source': 'root', 'target': f, 'label': 'contains', 
                                            'color': COLORS['primary_light']})
            if sf != tf:
                folder_links[(sf, tf)] += 1
    
    for (sf, tf), cnt in folder_links.items():
        struct_edges.append({'source': sf, 'target': tf, 'label': f'{cnt} links', 
                            'color': COLORS['primary_light'], 'weight': cnt})
    
    with open('_d3structure.json', 'w') as f:
        json.dump({'nodes': struct_nodes, 'links': struct_edges,
  "directed": True,
  "multigraph": True}, f, indent=2)
    print(f'✓ _d3structure.json: {len(struct_nodes)} nodes, {len(struct_edges)} edges')
else:
    print('No relationships available')

Extracted 209 relationships

✓ _d3graph.json: 112 nodes, 209 edges
✓ _d3structure.json: 17 nodes, 18 edges


## Preview with Sigma.js

In [8]:
try:
    from ipysigma import Sigma
    import networkx as nx
except ImportError:
    import sys
    !{sys.executable} -m pip install ipysigma networkx --quiet
    from ipysigma import Sigma
    import networkx as nx

def preview(json_file):
    data = json.load(open(json_file))
    G = nx.node_link_graph(data)
    print(f'{json_file}: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges')
    return Sigma(G, node_color='color', edge_color='color', height=700, start_layout=3)

for f in ['_d3structure.json', '_d3graph.json']:
    if os.path.exists(f):
        display(preview(f))

_d3structure.json: 17 nodes, 18 edges


Sigma(nx.MultiDiGraph with 17 nodes and 18 edges)

_d3graph.json: 112 nodes, 209 edges


Sigma(nx.MultiGraph with 112 nodes and 209 edges)